In [1]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time

In [22]:
# Get url by criteria in current_url below
def get_current_url(url, job_title, location):
    chrome_options = Options()
    chrome_options.add_argument("--incognito")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="text-input-what"]').send_keys(job_title)
    time.sleep(2)    
    
#     driver.find_element(By.XPATH, '//*[@id="text-input-where"]').send_keys(location)
    input_element = driver.find_element(By.XPATH, '//*[@id="text-input-where"]')
    input_element.send_keys(Keys.CONTROL, "a") # select all content
    input_element.send_keys(Keys.DELETE) # delete the content
#     time.sleep(1)
    input_element.send_keys(location)
        
    time.sleep(2)
    driver.find_element(By.XPATH, '/html/body/div').click()
    time.sleep(2)
    try:
        driver.find_element(By.XPATH, '//*[@id="jobsearch"]/button').click()
    except:
        driver.find_element(By.XPATH, '//*[@id="whatWhereFormId"]/div[3]/button').click()
    current_url = driver.current_url

    return current_url 

current_url = get_current_url('https://it.indeed.com/','Data Analyst',"Milano")   #"" is location
print(current_url)

https://it.indeed.com/jobs?q=Data%20Analyst&l=Milano%20Est%2C%20Lombardia&from=searchOnHP


In [23]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("window-size=1400,1400")
driver = webdriver.Chrome(options=chrome_options)

#PATH = "C://Program Files (x86)//chromedriver.exe"
#driver = webdriver.Chrome(PATH)

In [25]:
for i in range(0,1, 1):
#     driver.get('https://it.indeed.com/jobs?q=Data%20Analyst&l=Torino%20Piemonte&start='+str(i))
    driver.get(current_url+str(i))
    driver.implicitly_wait(5)

    jobtitles = []
    companies = []
    locations = []
    descriptions = []
    ratings = []
    job_types = []
    salaries = []
    dates = []



    jobs = driver.find_elements(By.CLASS_NAME, "slider_container")

    for job in jobs:

            jobtitle = job.find_element(By.CLASS_NAME, 'jobTitle').text.replace("new", "").strip()
            jobtitles.append(jobtitle)
            company = job.find_element(By.CLASS_NAME, 'companyName').text.replace("new", "").strip()
            companies.append(company)
            location = job.find_element(By.CLASS_NAME, 'companyLocation').text.replace("new", "").strip()
            locations.append(location)
            description = job.find_element(By.CLASS_NAME, 'job-snippet').text.replace("new", "").strip()
            descriptions.append(description)

            # RATING            
            try:
                rating_element = job.find_element(By.CLASS_NAME, 'ratingNumber')
                rating = rating_element.find_element(By.TAG_NAME, 'span').text
                ratings.append(rating)
            except:
                #print(f"No rating found for company: {company}")
                ratings.append(np.nan)

            # JOB TYPE                
            try:
                job_type_element = job.find_element(By.CLASS_NAME, 'attribute_snippet')
                job_type_aria = job_type_element.find_element(By.CSS_SELECTOR, '[aria-label="Job type"]')
                job_type_text = job_type_element.text.replace(job_type_aria.get_attribute('aria-label'), '').strip()
                job_types.append(job_type_text)
            except:
                job_types.append(np.nan)
                
            # SALARY                
            try:
                salary_element = job.find_element(By.CLASS_NAME, 'attribute_snippet')
                salary_aria = salary_element.find_element(By.CSS_SELECTOR, '[aria-label="Salary"]')
                salary_text = salary_element.text.replace(salary_aria.get_attribute('aria-label'), '').strip()
                salaries.append(salary_text)
            except:
                salaries.append(np.nan)
            
            #  PUBBLICATION DATE               
            try:
                date = job.find_element(By.CLASS_NAME, 'date').text.split('\n')[1]
                dates.append(date)
            except:
                dates.append(np.nan)

            
###########################                
            try:
                WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "button.popover-x-button-close.icl-CloseButton"))).click()
            except:
                pass



    df_da=pd.DataFrame()
    df_da['JobTitle']=jobtitles
    df_da['Company']=companies
    df_da['Location']=locations
    df_da['Description']=descriptions
    df_da['Ratings']= ratings
    df_da['Job_types']= job_types
    df_da['Salary']= salaries
    df_da['Pubblication_Date']= dates
#     print(df_da)

In [26]:
df_da
# estrarre la descrizione
# link????   
# df_da.to_excel('milano1st.xlsx', index=False)

,JobTitle,Company,Location,Description,Ratings,Job_types,Salary,Pubblication_Date
0,"Associate, Western Europe Financial Institutions",S&P Global,20121 Milano,Strong communication and interpersonal skills ...,NaN,NaN,NaN,30+ giorni fa
1,Data Analyst,Rovagnati,"Monza, Lombardia",Disegno e implementazione delle procedure di i...,"3,4",NaN,40.000 € - 50.000 € all'anno,Oggi
2,Data Engineer (hybrid-remote),Expandi Group,Da remoto in 20125 Milano,"Conoscenza di data orchestration tool, preferi...",NaN,NaN,NaN,Oggi
3,Business Analyst,Progresso,20090 Pieve Emanuele,Elaborazione e monitoraggio dei piani industri...,NaN,Tempo pieno\n+1,NaN,4 giorni fa
4,Data Analyst Jr,MetaDig,20099 Sesto San Giovanni,MetaDig* è un’azienda italiana protagonista ne...,NaN,Tempo pieno\n+2,NaN,Ultimo accesso 1 giorno fa
5,Finance & Administration Accountant,Hostmate s.r.l.,20125 Milano,Gestione relazione con il commercialista azien...,NaN,Tempo pieno,NaN,Ultimo accesso 1 giorno fa
6,SAS Data Analyst,UNIPOL GRUPPO,"Milano, Lombardia","Come parte integrante del Actuarial, Normative...","3,9",Tempo pieno,NaN,30+ giorni fa
7,Microsoft Business Intelligence Data Analyst s...,KA Resources,"Da remoto in Milano, Lombardia",La societá é attiva da piú di 50 anni e suppor...,NaN,Tempo pieno,NaN,30+ giorni fa
8,Data Analyst,Dolmen SRL,"Milano, Lombardia","Per importante cliente di Milano, ricerchiamo ...",NaN,NaN,NaN,3 giorni fa
9,Senior Business Intelligence Analyst,Sicuritalia,"Da remoto in Milano, Lombardia",Progettazione dello strato di presentazione se...,"2,9",Tempo pieno,NaN,30+ giorni fa


In [84]:
# Word Count and Frequencies

# Per sostituire i caratteri speciali con empty string
import string
def strip_puntuactions(line):
    for character in string.punctuation:
        line = line.replace(character, "")
    return line   

filepath = "filepath.txt"
word_count = {}

with open(filepath, 'r') as fi:
    for line in fi:
        line = strip_puntuactions(line)
        words = line.split()
        
        for word in words:
            word = word.lower()
            if word not in word_count:
                word_count[word] = 0
            word_count[word] += 1

In [86]:
list(word_count.keys())[:40]

['strong',
 'communication',
 'and',
 'interpersonal',
 'skills',
 'to',
 'represent',
 'sp',
 'as',
 'primary',
 'analyst',
 'in',
 'meetings',
 'willing',
 'travel',
 'sometimes',
 'at',
 'short',
 'notice',
 'disegno',
 'e',
 'implementazione',
 'delle',
 'procedure',
 'di',
 'integrazione',
 'dati',
 'dai',
 'sistemi',
 'alimentanti',
 'erp',
 'crm',
 'sfa',
 'mes',
 'wms',
 'demand',
 'planning',
 'conoscenza',
 'data',
 'orchestration']

## Estrarre ALL Programming Languages da wikip

In [49]:
r = requests.get("https://en.wikipedia.org/wiki/Timeline_of_programming_languages")
# Convert to a beautiful soup object
soup = bs(r.text, 'html.parser')



# table = soup.find_all("table", attrs={"class":"wikitable sortable"})
tables = soup.find_all("table", class_="wikitable sortable")

all_languages = pd.DataFrame()

# Iterate through each table, read it with Pandas and append it to the "all_languages" dataframe
for table in tables:
    df = pd.read_html(str(table))[0]
    all_languages = all_languages.append(df)
    
# Drop the rows where column names are repeated
all_languages = all_languages[~all_languages.iloc[:, 0].str.contains('Programming language|Year', na=False)]
all_languages.reset_index(drop=True, inplace=True)

C:\Users\marti\AppData\Local\Temp\ipykernel_26372\831950840.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_languages = all_languages.append(df)
C:\Users\marti\AppData\Local\Temp\ipykernel_26372\831950840.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_languages = all_languages.append(df)
C:\Users\marti\AppData\Local\Temp\ipykernel_26372\831950840.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_languages = all_languages.append(df)
C:\Users\marti\AppData\Local\Temp\ipykernel_26372\831950840.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_languages = all_languages.append(df)
C:\Users\marti\AppData\Local

In [78]:
# Visualization Tools
r = requests.get("https://hevodata.com/learn/data-analysis-tools/")
soup = bs(r.text, 'html.parser')

a_elements = soup.find_all("a", class_="rank-math-link")
viz_tools = []
for a in a_elements:
    viz_tools.append(a.text)
    
viz_tools = viz_tools[11:]

In [81]:
# Merge into single dataframe
viz_tools = pd.Series(viz_tools)
all_languages['Visualization Tools'] = viz_tools



In [82]:
all_languages

,Year,Name,"Chief developer, company",Predecessor(s),Visualization Tools
0,1804,Jacquard machine,Joseph Marie Jacquard,none (unique language),Power BI
1,1879,Begriffsschrift,Gottlob Frege,none (unique language),Tableau
2,1943–45,Plankalkül (year of conceptualization),Konrad Zuse,none (unique language),Qlik Sense
3,1943–46,ENIAC coding system,"John von Neumann, John Mauchly, J. Presper Eck...",none (unique language),ThoughtSpot
4,1946,ENIAC Short Code,Richard Clippinger and John von Neumann after ...,none (unique language),Oracle Analytics
...,...,...,...,...,...
393,2018,Fortran 2018,ISO/IEC JTC1/SC22/WG5 N2150:2018,Fortran 2008,NaN
394,2019,Bosque,"Mark Marron, Microsoft","JavaScript, TypeScript, ML",NaN
395,2020,C++20,C++ ISO/IEC 14882:2020,"C++, Standard C, C",NaN
396,2021,Microsoft Power Fx,"Vijay Mital, Robin Abraham, Shon Katzenberger,...",Excel formulas,NaN
